# Лабораторная работа 5. Деревья решений
## Классификация грибов

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo 
import math
import random
from collections import Counter

## 1. Загрузка и подготовка данных

In [ ]:
# Загрузка датасета
mushroom = fetch_ucirepo(id=73) 

# Данные
X = mushroom.data.features 
y = mushroom.data.targets 

# Преобразуем целевую переменную в бинарный формат (0 - edible, 1 - poisonous)
y_binary = (y['poisonous'] == 'p').astype(int)

# Обработка пропущенных значений
X = X.fillna('unknown')

print("Размерность данных:", X.shape)
print("\nПервые 5 строк:")
print(pd.concat([X.head(), y_binary.head()], axis=1))

## 2. Отбор признаков

In [ ]:
def select_features(X, method='sqrt'):
    """
    Отбор признаков для дерева решений
    """
    n_features = X.shape[1]
    
    if method == 'sqrt':
        n_selected = int(math.sqrt(n_features))
    else:
        n_selected = n_features
    
    selected_indices = random.sample(range(n_features), n_selected)
    selected_features = X.columns[selected_indices].tolist()
    
    print(f"Отобрано {n_selected} признаков из {n_features}:")
    print(selected_features)
    
    return selected_features

In [ ]:
# Отбор признаков
selected_features = select_features(X)
X_selected = X[selected_features].copy()

## 3. Реализация дерева решений

In [ ]:
class Node:
    """Узел дерева решений"""
    def __init__(self, feature=None, value=None, results=None, children=None):
        self.feature = feature      # Признак для разделения
        self.value = value          # Значение признака
        self.results = results      # Распределение классов (для листа)
        self.children = children or []  # Дочерние узлы

class DecisionTree:
    """Дерево решений для классификации"""
    
    def __init__(self, max_depth=10, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
    
    def calculate_gini(self, y):
        """Вычисление коэффициента Джини"""
        if len(y) == 0:
            return 0
        
        class_counts = Counter(y)
        gini = 1.0
        
        for count in class_counts.values():
            probability = count / len(y)
            gini -= probability ** 2
            
        return gini
    
    def split_data(self, X, y, feature, value):
        """Разделение данных по значению признака"""
        left_X, left_y = [], []
        right_X, right_y = [], []
        
        for i in range(len(X)):
            if X[i][feature] == value:
                left_X.append(X[i])
                left_y.append(y[i])
            else:
                right_X.append(X[i])
                right_y.append(y[i])
                
        return left_X, left_y, right_X, right_y
    
    def find_best_split(self, X, y, features):
        """Поиск лучшего разделения"""
        best_gini = float('inf')
        best_feature = None
        best_value = None
        best_splits = None
        
        current_gini = self.calculate_gini(y)
        
        for feature in features:
            # Получаем все уникальные значения признака
            values = set([x[feature] for x in X])
            
            for value in values:
                left_X, left_y, right_X, right_y = self.split_data(X, y, feature, value)
                
                if len(left_y) == 0 or len(right_y) == 0:
                    continue
                
                # Взвешенный коэффициент Джини
                left_weight = len(left_y) / len(y)
                right_weight = len(right_y) / len(y)
                
                weighted_gini = (left_weight * self.calculate_gini(left_y) + 
                               right_weight * self.calculate_gini(right_y))
                
                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    best_feature = feature
                    best_value = value
                    best_splits = (left_X, left_y, right_X, right_y)
        
        # Если улучшение незначительное, не разделяем
        if best_gini >= current_gini - 0.001:
            return None, None, None
            
        return best_feature, best_value, best_splits
    
    def build_tree(self, X, y, features, depth=0):
        """Рекурсивное построение дерева"""
        
        # Базовые случаи остановки
        if (len(y) < self.min_samples_split or 
            depth >= self.max_depth or 
            len(set(y)) == 1):
            
            return Node(results=Counter(y))
        
        # Поиск лучшего разделения
        feature, value, splits = self.find_best_split(X, y, features)
        
        if feature is None:
            return Node(results=Counter(y))
        
        left_X, left_y, right_X, right_y = splits
        
        # Рекурсивное построение дочерних узлов
        left_child = self.build_tree(left_X, left_y, features, depth + 1)
        right_child = self.build_tree(right_X, right_y, features, depth + 1)
        
        return Node(feature=feature, value=value, children=[left_child, right_child])
    
    def fit(self, X, y):
        """Обучение дерева"""
        # Преобразуем DataFrame в список словарей для удобства
        X_list = X.to_dict('records')
        y_list = y.tolist()
        
        features = list(X.columns)
        
        self.root = self.build_tree(X_list, y_list, features)
    
    def predict_single(self, x):
        """Предсказание для одного примера"""
        node = self.root
        
        while node.children:
            if x[node.feature] == node.value:
                node = node.children[0]  # Левый ребенок
            else:
                node = node.children[1]  # Правый ребенок
        
        # Возвращаем наиболее частый класс
        if node.results:
            return max(node.results.items(), key=lambda x: x[1])[0]
        return 0
    
    def predict(self, X):
        """Предсказание для набора данных"""
        X_list = X.to_dict('records')
        return [self.predict_single(x) for x in X_list]
    
    def predict_proba_single(self, x):
        """Вероятности классов для одного примера"""
        node = self.root
        
        while node.children:
            if x[node.feature] == node.value:
                node = node.children[0]
            else:
                node = node.children[1]
        
        if node.results:
            total = sum(node.results.values())
            proba_1 = node.results.get(1, 0) / total
            return [1 - proba_1, proba_1]
        return [0.5, 0.5]
    
    def predict_proba(self, X):
        """Вероятности классов для набора данных"""
        X_list = X.to_dict('records')
        probabilities = []
        
        for x in X_list:
            proba = self.predict_proba_single(x)
            probabilities.append(proba[1])  # Вероятность класса 1
        
        return probabilities

## 4. Обучение и оценка модели

In [ ]:
def train_test_split(X, y, test_size=0.3, random_state=None):
    """Разделение на обучающую и тестовую выборки"""
    if random_state is not None:
        np.random.seed(random_state)
    
    n = len(X)
    test_indices = np.random.choice(n, size=int(n * test_size), replace=False)
    train_indices = np.setdiff1d(np.arange(n), test_indices)
    
    X_train = X.iloc[train_indices]
    X_test = X.iloc[test_indices]
    y_train = y.iloc[train_indices]
    y_test = y.iloc[test_indices]
    
    return X_train, X_test, y_train, y_test

In [ ]:
def calculate_metrics(y_true, y_pred):
    """Вычисление метрик без использования библиотек"""
    tp = fp = tn = fn = 0
    
    for true, pred in zip(y_true, y_pred):
        if true == 1 and pred == 1:
            tp += 1
        elif true == 0 and pred == 1:
            fp += 1
        elif true == 0 and pred == 0:
            tn += 1
        elif true == 1 and pred == 0:
            fn += 1
    
    accuracy = (tp + tn) / (tp + fp + tn + fn) if (tp + fp + tn + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    return accuracy, precision, recall, tp, fp, tn, fn

In [ ]:
# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X_selected, y_binary, test_size=0.3, random_state=42)

print(f"Обучающая выборка: {len(X_train)} примеров")
print(f"Тестовая выборка: {len(X_test)} примеров")

# Обучение модели
tree = DecisionTree(max_depth=5, min_samples_split=10)
tree.fit(X_train, y_train)

# Предсказания
y_pred = tree.predict(X_test)
y_proba = tree.predict_proba(X_test)

# Оценка модели
accuracy, precision, recall, tp, fp, tn, fn = calculate_metrics(y_test.values, y_pred)

print("\nРезультаты оценки:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"\nМатрица ошибок:")
print(f"TP: {tp}, FP: {fp}")
print(f"FN: {fn}, TN: {tn}")

## 5. Построение кривых AUC-ROC и AUC-PR

In [ ]:
def calculate_auc_roc(y_true, y_proba):
    """Вычисление AUC-ROC без использования библиотек"""
    # Сортируем по убыванию вероятности
    data = sorted(zip(y_proba, y_true), key=lambda x: x[0], reverse=True)
    
    # Инициализация
    tpr_list = [0]
    fpr_list = [0]
    
    tp = fp = 0
    total_p = sum(y_true)
    total_n = len(y_true) - total_p
    
    last_prob = None
    
    for prob, true in data:
        if prob != last_prob:
            tpr = tp / total_p if total_p > 0 else 0
            fpr = fp / total_n if total_n > 0 else 0
            tpr_list.append(tpr)
            fpr_list.append(fpr)
            last_prob = prob
        
        if true == 1:
            tp += 1
        else:
            fp += 1
    
    # Добавляем конечную точку
    tpr_list.append(1)
    fpr_list.append(1)
    
    # Вычисление AUC методом трапеций
    auc = 0
    for i in range(1, len(tpr_list)):
        auc += (fpr_list[i] - fpr_list[i-1]) * (tpr_list[i] + tpr_list[i-1]) / 2
    
    return fpr_list, tpr_list, auc

In [ ]:
def calculate_auc_pr(y_true, y_proba):
    """Вычисление AUC-PR без использования библиотек"""
    # Сортируем по убыванию вероятности
    data = sorted(zip(y_proba, y_true), key=lambda x: x[0], reverse=True)
    
    precision_list = []
    recall_list = []
    
    tp = fp = 0
    total_p = sum(y_true)
    
    for prob, true in data:
        if true == 1:
            tp += 1
        else:
            fp += 1
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 1
        recall = tp / total_p if total_p > 0 else 0
        
        precision_list.append(precision)
        recall_list.append(recall)
    
    # Вычисление AUC методом трапеций
    auc_pr = 0
    for i in range(1, len(precision_list)):
        auc_pr += (recall_list[i] - recall_list[i-1]) * (precision_list[i] + precision_list[i-1]) / 2
    
    return precision_list, recall_list, auc_pr

In [ ]:
# Вычисление AUC-ROC
fpr, tpr, auc_roc = calculate_auc_roc(y_test.values, y_proba)

# Вычисление AUC-PR
precision_curve, recall_curve, auc_pr = calculate_auc_pr(y_test.values, y_proba)

print("Площади под кривыми:")
print(f"AUC-ROC: {auc_roc:.4f}")
print(f"AUC-PR: {auc_pr:.4f}")

In [ ]:
# Построение графиков
plt.figure(figsize=(12, 5))

# ROC-кривая
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, 'b-', label=f'ROC curve (AUC = {auc_roc:.4f})')
plt.plot([0, 1], [0, 1], 'r--', label='Random classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(True)

# PR-кривая
plt.subplot(1, 2, 2)
plt.plot(recall_curve, precision_curve, 'g-', label=f'PR curve (AUC = {auc_pr:.4f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Дополнительная информация о модели
print("\nДополнительная информация:")
print(f"Количество признаков: {len(selected_features)}")
print(f"Максимальная глубина дерева: {tree.max_depth}")
print(f"Минимальное количество samples для разделения: {tree.min_samples_split}")
print(f"Размер тестовой выборки: {len(X_test)} примеров")

# Распределение классов в тестовой выборке
class_dist = Counter(y_test)
print(f"\nРаспределение классов в тестовой выборке:")
print(f"Class 0 (edible): {class_dist[0]} примеров ({class_dist[0]/len(y_test)*100:.1f}%)")
print(f"Class 1 (poisonous): {class_dist[1]} примеров ({class_dist[1]/len(y_test)*100:.1f}%)")